<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo"  />
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:
- **TASK 1:** Mark all launch sites on a map
- **TASK 2:** Mark the success/failed launches for each site on the map
- **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [2]:
!pip3 install folium
!pip3 install wget

   ---------------------------------------- 0.0/100.0 kB ? eta -:--:--
   ---- ----------------------------------- 10.2/100.0 kB ? eta -:--:--
   ---------------------------------------- 100.0/100.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9680 sha256=aa008c4bb5706b7c180d0d70dd733cd84dede4c81464d67def6ffae2d6bea83f
  Stored in directory: c:\users\dell\appdata\local\pip\cache\wheels\40\b3\0f\a40dbd1c6861731779f62cc4babcb234387e11d697df70ee97
Successfully built wget


In [3]:
import folium
import wget
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site. 


In [5]:
import pandas as pd

# Define the file path
file_path = r'D:\06_CONTINUOUS LEARNING\01-DATA SCIENTIST\IBM Data Science-Coursera\10_Data Science Capstone Project\week03\spacex_df.csv'

# Load the CSV file into a DataFrame
spacex_df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame to confirm it's loaded correctly
print(spacex_df.head())


   Unnamed: 0  Flight Number        Date Time (UTC) Booster Version  \
0           0              1  2010-06-04   18:45:00  F9 v1.0  B0003   
1           1              2  2010-12-08   15:43:00  F9 v1.0  B0004   
2           2              3  2012-05-22    7:44:00  F9 v1.0  B0005   
3           3              4  2012-10-08    0:35:00  F9 v1.0  B0006   
4           4              5  2013-03-01   15:10:00  F9 v1.0  B0007   

   Launch Site                                            Payload  \
0  CCAFS LC-40               Dragon Spacecraft Qualification Unit   
1  CCAFS LC-40  Dragon demo flight C1, two CubeSats,  barrel o...   
2  CCAFS LC-40                             Dragon demo flight C2+   
3  CCAFS LC-40                                       SpaceX CRS-1   
4  CCAFS LC-40                                       SpaceX CRS-2   

   Payload Mass (kg)      Orbit         Customer        Landing Outcome  \
0                0.0        LEO           SpaceX  Failure   (parachute)   
1       

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example, 


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle. 


Now, let's add a circle for each launch site in data frame `launch_sites`


_TODO:_  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label


In [10]:
import folium
from folium.features import DivIcon

# Data for the launch sites (you might already have this in your DataFrame)
launch_sites_df = pd.DataFrame({
    'Launch Site': ['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
    'Lat': [28.562302, 28.563197, 28.573255, 34.632834],
    'Long': [-80.577356, -80.576820, -80.646895, -120.610745]
})

# Define initial map location and zoom level. Adjust the center to be more central to all sites
# Using average latitude and longitude from your locations might be a good start
average_lat = launch_sites_df['Lat'].mean()
average_long = launch_sites_df['Long'].mean()
site_map = folium.Map(location=[average_lat, average_long], zoom_start=4)

# Add circles and markers for each launch site
for index, row in launch_sites_df.iterrows():
    # Extract coordinates and launch site name
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    
    # Create a circle at the launch site coordinate with a popup label showing its name
    circle = folium.Circle(
        location=site_coordinate,
        radius=1000,  # Circle radius in meters
        color='#d35400',
        fill=True,
        popup=folium.Popup(site_name)
    )
    
    # Create a marker at the launch site coordinate with an icon showing its name
    marker = folium.map.Marker(
        location=site_coordinate,
        icon=DivIcon(
            icon_size=(20,20),
            icon_anchor=(0,0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{}</b></div>'.format(site_name)
        )
    )
    
    # Add the circle and marker to the map
    site_map.add_child(circle)
    site_map.add_child(marker)

# Display the map
site_map


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:
- Are all launch sites in proximity to the Equator line?
- Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records. 
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


_TODO:_ Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Assuming spacex_df is already loaded and contains the necessary data
# Extend spacex_df with a new column for marker colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# Initialize the map at a central coordinate
average_lat = spacex_df['Lat'].mean()
average_long = spacex_df['Long'].mean()
site_map = folium.Map(location=[average_lat, average_long], zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Add each launch as a marker to the cluster
for index, row in spacex_df.iterrows():
    site_coordinate = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    marker_color = row['marker_color']
    launch_outcome = 'Success' if row['class'] == 1 else 'Failure'

    # Create a marker for each launch and add to the marker cluster
    marker = folium.Marker(
        location=site_coordinate,
        popup=f'Launch Site: {site_name}\nOutcome: {launch_outcome}',
        icon=folium.Icon(color=marker_color)
    )
    marker_cluster.add_child(marker)

# Display the map
site_map


In [14]:
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


_TODO:_ For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
# e.g., icon=folium.Icon(color='white', icon_color=row['marker_color']
for index, record in spacex_df.iterrows():
    # TODO: Create and add a Marker cluster to the site map
    # marker = folium.Marker(...)
    marker_cluster.add_child(marker)

site_map

In [16]:
import folium
from folium.plugins import MarkerCluster

# Assuming spacex_df is already defined with the appropriate data as shown in the table
# You may need to define spacex_df here if not already defined

# Initialize map with the average coordinates from the spacex_df for a suitable initial view
average_lat = spacex_df['Lat'].mean()
average_long = spacex_df['Long'].mean()
site_map = folium.Map(location=[average_lat, average_long], zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster()

# Add the MarkerCluster to the map
site_map.add_child(marker_cluster)

# Iterate over the spacex_df DataFrame to create and add markers to the cluster
for index, record in spacex_df.iterrows():
    # Extract the latitude and longitude
    site_coordinate = [record['Lat'], record['Long']]
    
    # Create a folium Marker object, customized with the marker_color
    marker = folium.Marker(
        location=site_coordinate,
        # Popup with launch site and class information
        popup=folium.Popup(f"{record['Launch Site']} - {'Success' if record['class'] == 1 else 'Failure'}"),
        # Customize the icon according to the class of the launch
        icon=folium.Icon(color='white', icon_color=record['marker_color'])
    )
    
    # Add the marker to the MarkerCluster
    marker_cluster.add_child(marker)

# Display the map
site_map


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [17]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [18]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

_TODO:_ Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

_TODO:_ After obtained its coordinate, create a `folium.Marker` to show the distance


In [ ]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
# distance_marker = folium.Marker(
#    coordinate,
#    icon=DivIcon(
#        icon_size=(20,20),
#        icon_anchor=(0,0),
#        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
#        )
#    )

_TODO:_ Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
# lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

In [21]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Define the launch sites data
launch_sites = [
    {'name': 'KSC LC-39A', 'Lat': 28.573255, 'Long': -80.646895},
    {'name': 'CCAFS SLC-40', 'Lat': 28.563197, 'Long': -80.576820}
]

# Function to calculate the distance between two points on the Earth
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6373.0  # radius of the Earth in km

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

# Initialize map
site_map = folium.Map(location=[28.562302, -80.577356], zoom_start=15)

# Place launch site markers on the map
for site in launch_sites:
    folium.Marker(
        [site['Lat'], site['Long']],
        icon=folium.Icon(color='orange', icon='rocket', prefix='fa'),
        popup=site['name']
    ).add_to(site_map)

# Coordinates of the coastline point (example coordinates, replace with the actual ones)
coastline_lat = 28.562302
coastline_lon = -80.56756

# Select one launch site for the distance calculation (CCAFS SLC-40 in this case)
launch_site_lat = launch_sites[1]['Lat']
launch_site_lon = launch_sites[1]['Long']

# Calculate the distance to the coastline
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Add a marker on the closest coastline point on the map
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>%s KM</b></div>' % "{:10.2f}".format(distance_coastline)
    )
).add_to(site_map)

# Draw a line between the launch site and the coastline point
folium.PolyLine([[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]], color="blue").add_to(site_map)

# Display the map
site_map


Your updated map with distance line should look like the following screenshot:


_TODO:_ Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site


In [22]:
import folium
from folium.features import DivIcon
from math import sin, cos, sqrt, atan2, radians

# Initialize map on a specific launch site
launch_site_lat = 28.563197  # Replace with the latitude of your launch site
launch_site_lon = -80.576820  # Replace with the longitude of your launch site
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Assume I  have determined the points of interest and their coordinates
points_of_interest = {
    'closest_city': {'name': 'City', 'lat': 28.4017, 'lon': -80.6045},
    'closest_railway': {'name': 'Railway', 'lat': 28.5721, 'lon': -80.5852},
    'closest_highway': {'name': 'Highway', 'lat': 28.5623, 'lon': -80.5708},
    # Add more points as needed
}

# Function to calculate distance between two coordinates
# ... (Same as previously defined) ...

# Function to add a distance marker to the map
def add_distance_marker(map_obj, point, launch_site_coord, point_name):
    distance = calculate_distance(launch_site_coord[0], launch_site_coord[1], point['lat'], point['lon'])
    # Create and add a distance marker to the map
    folium.Marker(
        [point['lat'], point['lon']],
        icon=DivIcon(
            icon_size=(200,20),
            icon_anchor=(0,0),
            html=f'<div style="font-size: 12px; color:#d35400;"><b>{point_name}: {distance:.2f} KM</b></div>',
        )
    ).add_to(map_obj)
    # Draw a line between the launch site and the point
    folium.PolyLine([launch_site_coord, [point['lat'], point['lon']]], color="blue").add_to(map_obj)

# Place a marker on the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color='red', icon='rocket', prefix='fa'),
    popup='Launch Site'
).add_to(site_map)

# Use the function to add distance markers and lines for each point of interest
launch_site_coord = [launch_site_lat, launch_site_lon]
for point_name, point in points_of_interest.items():
    add_distance_marker(site_map, point, launch_site_coord, point_name)

# Display the map
site_map


After you plot distance lines to the proximities, you can answer the following questions easily:
- Are launch sites in close proximity to railways?
                                        - Raiways are approximately 1.28km distance
  
- Are launch sites in close proximity to highways?
                                        - Highways are approximately, 0.60km distance
  
- Are launch sites in close proximity to coastline?
                                        - Coastaline is approximately 0.91 km
  
- Do launch sites keep certain distance away from cities?
                                          - Cities are 18.17 ksm 
Also please try to explain your findings.

my findings can summarized as follows
1. global map highlighting the locations of SpaceX launch sites.
2. The map illustrates the launch site's location and measures the distances to the nearest railway and highway.
3. 
Distances are labeled on the map, showing the railway at 1.28 km and the highway at 0.60 km from the launch sit
4. The image displays a cluster of markers at a launch site, with green markers indicating successful launches and red markers indicating failures.
5. 
A coordinate display at the top right corner suggests the map has interactive features to show precise locations on mouse hover



e.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork865-2023-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2021-05-26|1.0|Yan|Created the initial version|


Copyright © 2021 IBM Corporation. All rights reserved.
